## Exporting to ONNX
You can export any Fireball model to ONNX using the [exportToOnnx](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.exportToOnnx) function. This notebook shows how to use this function to create an ONNX model. It assumes that a trained LeNet-5 model already exists in the ```Models``` directory. You can use the notebook [Handwritten Digit recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb) to create and train a LeNet-5 model.

Fireball can also export models with reduced number of parameters, pruned models, and quatized models. Please refer to the following notebooks for more information:

- [Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)
- [Pruning LeNet-5 Model](LeNet5-MNIST-Prune.ipynb)
- [Quantizing LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)

Note: Fireball uses the ```onnx``` python package to export models to ONNX. We also use the ```onnxruntime``` here to run and evaluate the onnx models.

## Load a pretrained model

In [1]:
from fireball import Model
from fireball.datasets.mnist import MnistDSet

testDs = MnistDSet.makeDatasets('test', batchSize=128)

# orgFileName = "Models/LeNet5.fbm"        # Original model
# orgFileName = "Models/LeNet5QR.fbm"      # Quantized - Retrained
# orgFileName = "Models/LeNet5PR.fbm"      # Pruned - Retrained
# orgFileName = "Models/LeNet5PRQR.fbm"    # Pruned - Retrained - Quantized - Retrained
# orgFileName = "Models/LeNet5RR.fbm"      # Reduced - Retrained
# orgFileName = "Models/LeNet5RRQR.fbm"    # Reduced - Retrained - Quantized - Retrained
# orgFileName = "Models/LeNet5RRPR.fbm"    # Reduced - Retrained - Pruned - Retrained
orgFileName = "Models/LeNet5RRPRQR.fbm"  # Reduced - Retrained - Pruned - Retrained - Quantized - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus='0')
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/LeNet5RRPRQR.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    111        
L2_CONV          14 14 6       KSP: 5 1 v, LR8          5 5 16        ReLU     MP(KSP):2 2 v    844        
L3_FC            5 5 16        LR8                      120           ReLU                      2,441      
L4_FC            120           LR8                      84            ReLU                      987        
L5_FC            84                                     10            None                      539        
OUT_CLASS        10            10 cla

## Export the model and check the exported ONNX model

In [2]:
onnxFileName = orgFileName.replace(".fbm",".onnx")
model.exportToOnnx(onnxFileName, runQuantized=True, classNames=[str(i) for i in range(10)], 
                   modelDocStr="Fireball example: LeNet-5 Model")

# Check the exported model. This throws exceptions if something is wrong with the exported model.
import onnx
from onnx import shape_inference

onnxModel = onnx.load(onnxFileName)
onnx.checker.check_model(onnxModel)



Exporting to ONNX model "Models/LeNet5RRPRQR.onnx" ... 
    Processed all 7 layers.                                     
    Saving to "Models/LeNet5RRPRQR.onnx" ... Done.
Done (0.03 Sec.)


## Using netron to visualize the exported model
We can now visualize the model's network structure using the [netron](https://github.com/lutzroeder/netron) package.

In [3]:
import netron
import platform

if platform.system() == 'Darwin':      # Running on MAC
    netron.start(onnxFileName)   
else:
    import socket
    hostIp = socket.gethostbyname(socket.gethostname())
    netron.start(onnxFileName, address=(hostIp,8084))

Serving 'Models/LeNet5RRPRQR.onnx' at http://localhost:8080


## Running inference using the exported ONNX model
Here we load the test dataset and use it to run inference on random samples from this dataset. We could use fireball's implementation of MNIST dataset, but here we want to run and test the ONNX model without any dependency to Fireball libraty.

In [4]:
# Load MNIST Test Dataset:
import numpy as np
import struct

def getDataset(imagesFileName, labelsFileName):
    file = open(imagesFileName, mode='rb')
    header = file.read(16)
    magic, numImages, imageWidth, imageHeight = struct.unpack(">iiii", header)
    assert magic == 2051, "Error: Invalid MNIST Image format!"

    buf = file.read(imageWidth * imageHeight * numImages)
    data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    data = (data-127.5)/255.0   # Normalize to [-1..1]
    samples = data.reshape(numImages, imageWidth, imageHeight, 1)

    file = open(labelsFileName, mode='rb')
    header = file.read(8)
    magic, numLabels = struct.unpack(">ii", header)
    assert magic == 2049, "Error: Invalid MNIST Label format!"

    buf = file.read(numLabels)
    labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return samples, labels

# Update the file names to point to the location of MNIST dataset
testSamples, testLabels = getDataset('/Users/shahab/data/mnist/t10k-images.idx3-ubyte',
                                     '/Users/shahab/data/mnist/t10k-labels.idx1-ubyte')

testSamples = testSamples.reshape(-1,1,28,28)

The code in the following cell shows how to run inference using an ONNX model. We get a random test sample and run prediction on the ONNX model. The ```argmax``` function gives us the actual label. You can run it several time to test it with different samples.

In [5]:
# Inference using the ONNX model and "onnxruntime"
import onnxruntime as ort
session = ort.InferenceSession(onnxModel.SerializeToString(), None)

i = np.random.randint(len(testLabels))
print( "Actual label of the sample no %d in the dataset: %d"%(i, testLabels[i]))
sample = testSamples[i:i+1]

y = session.run(['ClassProbs'],{'InputImage':sample})
print( "Predicted label: %d"%(np.argmax(y[0])))

Actual label of the sample no 1071 in the dataset: 7
Predicted label: 7


Now we can run the evaluation over all test samples.

In [6]:
predictedLabels = [np.argmax( session.run(['ClassProbs'],{'InputImage':[sample]})[0]) for sample in testSamples]
accuracy = float(np.sum(testLabels == predictedLabels))/len(testLabels)
print( "Test Accuracy: %f"%(accuracy) )

Test Accuracy: 0.989100


## Where do I go from here?

[Exporting LeNet-5 Model to CoreML](LeNet5-MNIST-CoreML.ipynb)

[Exporting LeNet-5 Model to TensorFlow](LeNet5-MNIST-TF.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Handwritten Digit Recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb)

[Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)

[Pruning LeNet-5 Model](LeNet5-MNIST-Prune.ipynb)

[Quantizing LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)